In [1]:
from keras.models import Sequential , Model
from keras.layers import Dense ,  BatchNormalization , Reshape , Input , Flatten
from keras.layers import Conv2D , MaxPool2D , Conv2DTranspose , UpSampling2D , ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Activation
from keras.layers import Dropout

from keras.initializers import truncated_normal , random_normal , constant

from keras.optimizers import Adam , RMSprop

from keras.datasets import mnist

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import os

import matplotlib as plt
import numpy as np
import keras.backend as K

import gc

%matplotlib inline

In [3]:
WIDTH = 96
HEIGHT = 96
CHANNEL = 3

LATENT_DIM = 100 #latent variable z sample from normal distribution

BATCH_SIZE = 64
EPOCHS = 10

PATH = 'faces/'

#生成多少个图像 长*宽
ROW = 5
COL = 5

#为WGAN增加的
N_CRITIC = 5 #训练G时使用
CLIP_VALUE = 0.01 #更新G的权重参数时进行截断使用


In [4]:

load_index = 0

images_name = os.listdir(PATH)

IMAGES_COUNT = len(images_name)


In [5]:
'''
(X_train , y_train),(X_test , y_test) = mnist.load_data()
X_train = X_train/127.5-1
X_train = np.expand_dims(X_train , 3)
'''

'\n(X_train , y_train),(X_test , y_test) = mnist.load_data()\nX_train = X_train/127.5-1\nX_train = np.expand_dims(X_train , 3)\n'

In [6]:
'''
def load_mnist():
    return X_train[np.random.randint(0, X_train.shape[0], BATCH_SIZE)]
    
def write_image_mnist(epoch):
    
    noise = np.random.normal(size = (ROW*COL , LATENT_DIM))
    generated_image = generator_i.predict(noise)
    generated_image = generated_image*0.5+0.5
    
    fig , axes = plt.pyplot.subplots(ROW , COL)
    
    count=0
    
    for i in range(ROW):
        for j in range(COL):
            axes[i][j].imshow(generated_image[count,:,:,0] , cmap = 'gray')
            axes[i][j].axis('off')
            count += 1
            
    fig.savefig('mnist_wgan/No.%d.png' % epoch)
    plt.pyplot.close()

'''

"\ndef load_mnist():\n    return X_train[np.random.randint(0, X_train.shape[0], BATCH_SIZE)]\n    \ndef write_image_mnist(epoch):\n    \n    noise = np.random.normal(size = (ROW*COL , LATENT_DIM))\n    generated_image = generator_i.predict(noise)\n    generated_image = generated_image*0.5+0.5\n    \n    fig , axes = plt.pyplot.subplots(ROW , COL)\n    \n    count=0\n    \n    for i in range(ROW):\n        for j in range(COL):\n            axes[i][j].imshow(generated_image[count,:,:,0] , cmap = 'gray')\n            axes[i][j].axis('off')\n            count += 1\n            \n    fig.savefig('mnist_wgan/No.%d.png' % epoch)\n    plt.pyplot.close()\n\n"

In [7]:

def load_image(batch_size = BATCH_SIZE):
    global load_index
    
    images = []
    
    for i in range(batch_size):
        images.append(plt.image.imread(PATH + images_name[(load_index + i) % IMAGES_COUNT]))
    
    load_index += batch_size
    
    return np.array(images)/127.5-1

def write_image(epoch):
    
    noise = np.random.normal(size = (ROW*COL , LATENT_DIM))
    generated_image = generator_i.predict(noise)
    generated_image = (generated_image+1)*127.5
    
    fig , axes = plt.pyplot.subplots(ROW , COL)
    
    count=0
    
    for i in range(ROW):
        for j in range(COL):
            axes[i][j].imshow(generated_image[count])
            axes[i][j].axis('off')
            count += 1
            
    fig.savefig('generated_faces_wgan/No.%d.png' % epoch)
    plt.pyplot.close()
    
    #plt.image.imsave('images/'+str(epoch)+'.jpg')


In [8]:
def conv2d(output_size):
    return Conv2D(output_size , kernel_size=(5,5) , strides=(2,2) , padding='same' , kernel_initializer=truncated_normal(stddev=0.02) , bias_initializer=constant(0.0))

def dense(output_size):
    return Dense(output_size , kernel_initializer=random_normal(stddev=0.02) , bias_initializer=constant(0.0))

def deconv2d(output_size):
    return Conv2DTranspose(output_size , kernel_size=(5,5) , strides=(2,2) , padding='same' , kernel_initializer=random_normal(stddev=0.02) , bias_initializer=constant(0.0))

def batch_norm():
    return BatchNormalization(momentum=0.9 , epsilon=1e-5)


In [9]:
def generator():
    #sample from noise z
    model = Sequential(name='generator')
    
    #cartoon 图像使用 96*96*3
    model.add(Dense(6*6*8*64 , input_shape=(LATENT_DIM,) , kernel_initializer=random_normal(stddev=0.02) , bias_initializer=constant(0.0)))
    
    model.add(Reshape((6, 6, 64*8)))
    
    model.add(batch_norm())
    model.add(Activation('relu'))

    model.add(deconv2d(64*4))
    model.add(batch_norm())
    model.add(Activation('relu'))
    
    model.add(deconv2d(64*2))
    model.add(batch_norm())
    model.add(Activation('relu'))
    
    model.add(deconv2d(64*1))
    model.add(batch_norm())
    model.add(Activation('relu'))
    
    model.add(deconv2d(3))
    model.add(Activation('tanh'))
    
    model.summary()
    
    noise = Input(shape=(LATENT_DIM , ) , name='input1')
    image = model(noise)
    
    return Model(noise , image , name='generator_Model')

In [10]:
def critic():
    #input a image to discriminate real or fake
    model = Sequential(name='critic')
    
    model.add(Conv2D(filters=64 , kernel_size=(5,5) , strides=(2,2) , padding='same' , input_shape=(WIDTH , HEIGHT , CHANNEL) , kernel_initializer=truncated_normal(stddev=0.02) , bias_initializer=constant(0.0) , name='conv1'))
    model.add(LeakyReLU(0.2))
    
    model.add(conv2d(64*2))
    model.add(batch_norm())
    model.add(LeakyReLU(0.2))
    
    model.add(conv2d(64*4))
    model.add(batch_norm())  
    model.add(LeakyReLU(0.2))

    
    model.add(conv2d(64*8))
    model.add(batch_norm())  
    model.add(LeakyReLU(0.2))

    model.add(Flatten())
    
    #===
    #如果没有下面的两个FC层 训练时发生损失不下降 且生成不出图像
    #model.add(dense(1024))
    #model.add(LeakyReLU(0.2))
    
    model.add(dense(256))
    model.add(LeakyReLU(0.2))
    #===
    model.add(dense(1))
    
    model.summary()
    
    image = Input(shape=(WIDTH , HEIGHT , CHANNEL) , name='input1')
    validity = model(image)
    
    return Model(image , validity , name='critic')

In [11]:
rmsprop = RMSprop(lr=0.00005)

def wgan_loss(y_true , y_pred):
    return K.mean(y_true*y_pred)

In [12]:
critic_i = critic()
critic_i.compile(optimizer=rmsprop , loss=wgan_loss , metrics=['accuracy'])

generator_i = generator()

z = Input(shape=(LATENT_DIM , ) , name='z')
image = generator_i(z)
validity = critic_i(image)

combined_model_i = Model(z , validity)
combined_model_i.compile(optimizer=rmsprop , loss=wgan_loss)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 48, 48, 64)        4864      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 128)       204928    
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 128)       512       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 24, 24, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 256)       819456    
_________________________________________________________________
batch_normalization_2 (Batch (None, 12, 12, 256)       1024      
__________

In [13]:
real_labels = -np.ones(shape=(BATCH_SIZE , 1)) #真实样本label为1
fake_labels = np.ones(shape=(BATCH_SIZE , 1)) #假样本label为0

for i in range(1000):
    #============================
    #训练一次G就要训练N_CRITIC次D（Discriminator）
    #===
    critic_i.trainable = True
    for layer in critic_i.layers:
        layer.trainable = True
    #===
    for _ in range(N_CRITIC):
        
        noise = np.random.normal(size=(BATCH_SIZE , LATENT_DIM))

        real_image = load_image()
        #训练判别器
        fake_image = generator_i.predict(noise)

        real_loss = critic_i.train_on_batch(real_image , real_labels)
        fake_loss = critic_i.train_on_batch(fake_image , fake_labels)

        loss = np.add(real_loss , fake_loss)/2
        
        #addin
        #WGAN的变化 对权重参数进行裁剪
        for layer in critic_i.layers:
            weights = layer.get_weights()
            weights = [np.clip(w , - CLIP_VALUE , CLIP_VALUE) for w in weights]
            layer.set_weights(weights)
    #============================
    #===
    critic_i.trainable = False
    for layer in critic_i.layers:
        layer.trainable = False
    #===
    
    #训练生成器
    noise2 = np.random.normal(size=(BATCH_SIZE , LATENT_DIM))
    generator_loss = combined_model_i.train_on_batch(noise2 , real_labels)

    print('epoch:%d loss:%f gene_loss:%f' % (i , 1-loss[0] , 1-generator_loss))

    if i % 10 == 0:
        write_image(i)
    
write_image(999)


C:\Anaconda3\lib\site-packages\keras\engine\training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 loss:1.001127 gene_loss:1.000315
epoch:1 loss:0.999923 gene_loss:1.008464
epoch:2 loss:0.998405 gene_loss:1.016632
epoch:3 loss:0.996507 gene_loss:1.025243
epoch:4 loss:0.995589 gene_loss:1.032789
epoch:5 loss:0.996031 gene_loss:1.037650
epoch:6 loss:0.997686 gene_loss:1.039668
epoch:7 loss:1.002765 gene_loss:1.038513
epoch:8 loss:1.008644 gene_loss:1.032182
epoch:9 loss:1.020407 gene_loss:1.020413
epoch:10 loss:1.036939 gene_loss:1.004989
epoch:11 loss:1.054953 gene_loss:0.989337
epoch:12 loss:1.072437 gene_loss:0.969723
epoch:13 loss:1.094888 gene_loss:0.951461
epoch:14 loss:1.118298 gene_loss:0.930042
epoch:15 loss:1.140145 gene_loss:0.908196
epoch:16 loss:1.169432 gene_loss:0.890403
epoch:17 loss:1.203541 gene_loss:0.868130
epoch:18 loss:1.237040 gene_loss:0.851855
epoch:19 loss:1.275418 gene_loss:0.839845
epoch:20 loss:1.317360 gene_loss:0.850781
epoch:21 loss:1.356701 gene_loss:0.822613
epoch:22 loss:1.406108 gene_loss:0.803189
epoch:23 loss:1.453833 gene_loss:0.730712
ep

epoch:190 loss:3.781750 gene_loss:-1.784286
epoch:191 loss:3.784108 gene_loss:-1.790191
epoch:192 loss:3.782423 gene_loss:-1.781964
epoch:193 loss:3.785064 gene_loss:-1.790474
epoch:194 loss:3.784670 gene_loss:-1.788048
epoch:195 loss:3.786098 gene_loss:-1.792413
epoch:196 loss:3.782448 gene_loss:-1.791641
epoch:197 loss:3.785310 gene_loss:-1.792556
epoch:198 loss:3.789949 gene_loss:-1.795129
epoch:199 loss:3.783939 gene_loss:-1.789996
epoch:200 loss:3.788069 gene_loss:-1.792017
epoch:201 loss:3.788654 gene_loss:-1.796126
epoch:202 loss:3.787542 gene_loss:-1.795836
epoch:203 loss:3.787554 gene_loss:-1.793746
epoch:204 loss:3.787971 gene_loss:-1.796203
epoch:205 loss:3.789522 gene_loss:-1.796383
epoch:206 loss:3.789460 gene_loss:-1.794634
epoch:207 loss:3.786931 gene_loss:-1.793903
epoch:208 loss:3.790461 gene_loss:-1.789697
epoch:209 loss:3.788538 gene_loss:-1.782940
epoch:210 loss:3.789866 gene_loss:-1.781554
epoch:211 loss:3.791476 gene_loss:-1.776756
epoch:212 loss:3.791499 gene_los

epoch:377 loss:3.807688 gene_loss:-1.822373
epoch:378 loss:3.809383 gene_loss:-1.819719
epoch:379 loss:3.810994 gene_loss:-1.821356
epoch:380 loss:3.807566 gene_loss:-1.813003
epoch:381 loss:3.807838 gene_loss:-1.824113
epoch:382 loss:3.810173 gene_loss:-1.822834
epoch:383 loss:3.810127 gene_loss:-1.820177
epoch:384 loss:3.809260 gene_loss:-1.822213
epoch:385 loss:3.807931 gene_loss:-1.823729
epoch:386 loss:3.810270 gene_loss:-1.823319
epoch:387 loss:3.809247 gene_loss:-1.823682
epoch:388 loss:3.810556 gene_loss:-1.823124
epoch:389 loss:3.810388 gene_loss:-1.821549
epoch:390 loss:3.811195 gene_loss:-1.823811
epoch:391 loss:3.808638 gene_loss:-1.822421
epoch:392 loss:3.812251 gene_loss:-1.824443
epoch:393 loss:3.809811 gene_loss:-1.820828
epoch:394 loss:3.808287 gene_loss:-1.823893
epoch:395 loss:3.811481 gene_loss:-1.825128
epoch:396 loss:3.810811 gene_loss:-1.818863
epoch:397 loss:3.809837 gene_loss:-1.823882
epoch:398 loss:3.812311 gene_loss:-1.823926
epoch:399 loss:3.810484 gene_los

epoch:564 loss:3.813353 gene_loss:-1.825067
epoch:565 loss:3.812702 gene_loss:-1.822130
epoch:566 loss:3.813928 gene_loss:-1.825066
epoch:567 loss:3.813564 gene_loss:-1.824383
epoch:568 loss:3.810439 gene_loss:-1.819223
epoch:569 loss:3.813519 gene_loss:-1.822903
epoch:570 loss:3.810557 gene_loss:-1.816012
epoch:571 loss:3.813372 gene_loss:-1.821332
epoch:572 loss:3.811710 gene_loss:-1.821246
epoch:573 loss:3.806470 gene_loss:-1.813350
epoch:574 loss:3.811640 gene_loss:-1.819893
epoch:575 loss:3.811033 gene_loss:-1.819490
epoch:576 loss:3.811943 gene_loss:-1.819165
epoch:577 loss:3.810019 gene_loss:-1.818850
epoch:578 loss:3.810801 gene_loss:-1.818923
epoch:579 loss:3.810157 gene_loss:-1.817741
epoch:580 loss:3.809667 gene_loss:-1.813039
epoch:581 loss:3.809642 gene_loss:-1.816327
epoch:582 loss:3.808823 gene_loss:-1.814627
epoch:583 loss:3.808841 gene_loss:-1.812648
epoch:584 loss:3.807222 gene_loss:-1.809823
epoch:585 loss:3.808275 gene_loss:-1.806891
epoch:586 loss:3.805264 gene_los

epoch:751 loss:3.813212 gene_loss:-1.825137
epoch:752 loss:3.815205 gene_loss:-1.824410
epoch:753 loss:3.814738 gene_loss:-1.827228
epoch:754 loss:3.814690 gene_loss:-1.827876
epoch:755 loss:3.814699 gene_loss:-1.822277
epoch:756 loss:3.816057 gene_loss:-1.828752
epoch:757 loss:3.816040 gene_loss:-1.826919
epoch:758 loss:3.814242 gene_loss:-1.828080
epoch:759 loss:3.816775 gene_loss:-1.828597
epoch:760 loss:3.813683 gene_loss:-1.822634
epoch:761 loss:3.816525 gene_loss:-1.829097
epoch:762 loss:3.815694 gene_loss:-1.826672
epoch:763 loss:3.816930 gene_loss:-1.828803
epoch:764 loss:3.815432 gene_loss:-1.824244
epoch:765 loss:3.817860 gene_loss:-1.829590
epoch:766 loss:3.816342 gene_loss:-1.829010
epoch:767 loss:3.817561 gene_loss:-1.829403
epoch:768 loss:3.816169 gene_loss:-1.826402
epoch:769 loss:3.816700 gene_loss:-1.830076
epoch:770 loss:3.817248 gene_loss:-1.827876
epoch:771 loss:3.815344 gene_loss:-1.829096
epoch:772 loss:3.817567 gene_loss:-1.828344
epoch:773 loss:3.817045 gene_los

epoch:938 loss:3.819560 gene_loss:-1.830842
epoch:939 loss:3.816247 gene_loss:-1.828218
epoch:940 loss:3.818858 gene_loss:-1.830251
epoch:941 loss:3.817608 gene_loss:-1.829159
epoch:942 loss:3.815171 gene_loss:-1.826324
epoch:943 loss:3.817466 gene_loss:-1.825917
epoch:944 loss:3.816676 gene_loss:-1.821595
epoch:945 loss:3.813273 gene_loss:-1.818489
epoch:946 loss:3.814590 gene_loss:-1.817526
epoch:947 loss:3.813235 gene_loss:-1.814858
epoch:948 loss:3.812843 gene_loss:-1.812361
epoch:949 loss:3.811794 gene_loss:-1.810277
epoch:950 loss:3.811245 gene_loss:-1.808395
epoch:951 loss:3.809380 gene_loss:-1.804917
epoch:952 loss:3.809881 gene_loss:-1.805589
epoch:953 loss:3.809750 gene_loss:-1.807615
epoch:954 loss:3.809262 gene_loss:-1.809164
epoch:955 loss:3.810415 gene_loss:-1.808777
epoch:956 loss:3.811758 gene_loss:-1.807642
epoch:957 loss:3.812791 gene_loss:-1.808295
epoch:958 loss:3.810825 gene_loss:-1.810253
epoch:959 loss:3.814791 gene_loss:-1.810650
epoch:960 loss:3.813832 gene_los

In [70]:
real_labels.shape

(64, 1)

In [53]:
discriminator_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input1 (InputLayer)          (None, 28, 28, 1)         0         
_________________________________________________________________
discriminator (Sequential)   (None, 1)                 533505    
Total params: 533,505
Trainable params: 0
Non-trainable params: 533,505
_________________________________________________________________


In [54]:
generator_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input1 (InputLayer)          (None, 100)               0         
_________________________________________________________________
generator (Sequential)       (None, 28, 28, 1)         1097744   
Total params: 1,097,744
Trainable params: 1,095,184
Non-trainable params: 2,560
_________________________________________________________________


In [39]:
combined_model_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
z (InputLayer)               (None, 100)               0         
_________________________________________________________________
generator_Model (Model)      (None, 96, 96, 3)         29029120  
_________________________________________________________________
discriminator_Model (Model)  (None, 1)                 14320641  
Total params: 43,349,761
Trainable params: 29,025,536
Non-trainable params: 14,324,225
_________________________________________________________________


In [ ]:
gc.collect()

In [14]:
modeli = Sequential()

modeli.add(Dense(128 * 7 * 7, activation="relu", input_shape=(LATENT_DIM,)))
modeli.add(Reshape((7, 7, 128)))
modeli.add(UpSampling2D())
modeli.add(Conv2D(128, kernel_size=3, padding="same"))
modeli.add(BatchNormalization(momentum=0.8))
modeli.add(Activation("relu"))
modeli.add(UpSampling2D())
modeli.add(Conv2D(64, kernel_size=3, padding="same"))
modeli.add(BatchNormalization(momentum=0.8))
modeli.add(Activation("relu"))
modeli.add(Conv2D(CHANNEL, kernel_size=3, padding="same"))
modeli.add(Activation("tanh"))

In [15]:
modeli.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_3 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 128)       0         
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, 28, 28, 128)       0         
__________

In [41]:
32*400

12800

In [ ]:
gc.collect()